In [25]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

from transformers import T5Tokenizer, T5ForConditionalGeneration

In [10]:
from torch import cuda

In [33]:
device = 'cuda' if cuda.is_available() else 'cpu'
LEARNING_RATE = 0.001

In [12]:
device

'cpu'

In [13]:
def QARetrieval(Dataset):
    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.source_len = source_len
        self.summ_len = summ_len
        self.text = self.data[output_col_name]
        self.ctext = self.data[context_col_name]
        
    def __len__(self):
        return len(self.text)
    
    def __getitem__(self, index):
        ctext = str(self.ctext[index])
        ctext = ' '.join(ctext.split())
        
        text = str(self.text[index])
        text = ' '.join(text.split())
        
        source = self.tokenizer.batch_encode_plus([ctext], max_length=self.source_len, pad_to_max_length=True, return_tensors='pt')
        target = self.tokenizer.batch_encode_plus([text], max_length=self.summ_len, pad_to_max_length=True, return_tensors='pt')
        
        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()
        
        return {
            'source_ids': source_ids.to(dtype=torch.long),
            'source_mask': source_mask.to(dtype=torch.long),
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }
        
        

In [18]:
def train(epoch, tokenizer, model,  device, loader, optimizer):
    model.train()
    for index, data in enumerate(loader, 0):
        y = data['target_ids'].to(device, type = torch.long)
        y_ids = y[l:, : - 1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data['source_ids'].to(device, type = torch.long)
        mask = data['source_mask'].to(device, dtype = torch.long)
        loss = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels = lm_labels)[0]
        if index%10 == 0:
            print({"Training Loss": loss.item(), "Time Check":str(datetime.now())})
        elif index%500 == 0:
            print(f'Epoch: {epoch}, Loss: {loss.item()}')
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [22]:
def validate(epoch, tokenizer, model, device, loader):
    model.eval()
    predictions = []
    actuals = []
    
    with torch.no_grad():
        for index, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source _mask'].to(device, dtype = torch.long)
            
            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask,
                max_length=150,
                num_beams=2,
                repetition_penalty= 2.5,
                length_penalty=1.0,
                early_stopping=True
            )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            if index%100==0:
                print (f'Completed {index}')
            predictions.extend(preds)
            actuals.extend(target)
    return predictions, actuals

In [27]:
model = T5ForConditionalGeneration.from_pretrained('t5-base')

Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [28]:
model = model.to(device)

In [34]:
optimizer = torch.optim.Adam(params = model.parameters(), lr = LEARNING_RATE)

In [37]:
training_set = QARetrieval(train_dataset, tokenizer, MAX_LEN, SUMMARY_LEN)

NameError: name 'train_dataset' is not defined